In [1]:
import pandas as pd
import numpy as np
import subprocess
import sys
sys.path.append("..")
from src.useful import *

## Parse HMM Search Output

In [ ]:
columns = ['target_name', 'target_accession', 'query_name', 'query_accession', 'full_sequence_evalue', 'full_sequence_score', 'full_sequence_bias', 'domain_e_evalue', 'domain_score', 'domain_bias', 'domain#est_exp', 'domain#est_reg', 'domain#est_clu', 'domain#est_ov', 'domain#est_env', 'domain#est_dom', 'domain#est_rep', 'domain#est_inc', 'description_of_target']
root_path = '../../data/results/hmmer_searches/'
all_bins_hmmer_fp = root_path + 'PET_search.hmmsearch_out'

sep='\s+'
df = pd.read_csv(all_bins_hmmer_fp, sep=sep, skiprows=3, skipfooter=10, header=None, names=columns, index_col=False, engine='python')

In [ ]:
#Make a dive_core_horizon column to eventually add on temperature data for each dive 
df['dive'] = df['target_name'].str.split('_').str[0:3].str.join('_')

#Read in fasta file and add columns for protein sequence and protein sequence length
guaymas2020_bins_fp = 'Guaymas2020bins_all.faa'
guaymas2020_bins_df = fasta2df(guaymas2020_bins_fp)
guaymas2020_bins_df['aa_sequence_length'] = guaymas2020_bins_df['aa_sequence'].str.len()

#Join aa_sequence and aa_sequence column to df from guaymas2020_bins_df using target_name as key
df = df.merge(guaymas2020_bins_df, left_on='target_name', right_on='ProteinID', how='left')


In [ ]:
df['dive'].unique()
#Build dictionary that connects each dive to its corresponding temperature
dive_temps = {'D4993_C5_LG':'35' , 'D4993_C5_H2': '72', 'D4993_C5_H1':'29', 'D4993_C5_H3':'33',
       'D4994_C39_H1':'89', 'D4994_C39_H2':'99', 'D4998_C1112_H1':'80', 'D4998_C2223_H2':'15',
       'D4998_C1112_H2':'100', 'D4991_C11_H1':'10', 'D4993_C5_H4': '39', 'D4998_C2223_H1':'4',
       'D4998_C1112_H3':'115'}

#Add temperature column to df
df['temperature'] = df['dive'].map(dive_temps)

In [ ]:
#Write out fasta files for each protein
pet_fp = '../../data/results/PETHits_Guaymas2020_ALLBINS.fasta'
with open(pet_fp, 'w') as f:
    for seq in df[['target_name', 'aa_sequence']].values:
        f.write('>' + seq[0] + '\n')
        f.write(seq[1] + '\n')
    f.close()

## Sort by e-value, temperature and de-duplicate hits

In [ ]:
#Rearrange so that the lowest e-value is at the top
pet_new = df.sort_values(by=['full_sequence_evalue'], ascending=True)

#Find duplicates that have the same aa_sequence and keep the one with the highest temperature
pet_new = pet_new.sort_values(by=['temperature'], ascending=False)
pet_new = pet_new.drop_duplicates(subset=['aa_sequence'], keep='first')
#pet_new.head(200).tail(50)

In [ ]:
#Write deduplicated pet_new to fasta files to save for later use
pet_fp = '../../data/results/PETHits_deduplicated_Guaymas2020_ALLBINS.fasta'
with open(pet_fp, 'w') as f:
    for seq in pet_new[['target_name', 'aa_sequence']].values:
        f.write('>' + seq[0] + '\n')
        f.write(seq[1] + '\n')
    f.close()

### Add on Phylogeny metadata 

In [ ]:
#Add phylogenetic annotation to pet_new
#Read in phylogenetic annotations
arc_fp = 'gtdbtk.ar53.summary.tsv'
bac_fp = 'gtdbtk.bac120.summary.tsv'
arc_df = pd.read_csv(arc_fp, sep='\t')
bac_df = pd.read_csv(bac_fp, sep='\t')
arc_df.head()

In [13]:
arc = arc_df[['user_genome', 'classification']]
bac = bac_df[['user_genome', 'classification']]
#arc['Domain'] = arc['classification'].str.split(';').str[0].str.split('__').str[1]
#bac['Domain'] = bac['classification'].str.split(';').str[0].str.split('__').str[1]

In [14]:
#Split classification by ; and pivot so that each level of classification is a column
arc = arc.join(arc['classification'].str.split(';', expand=True).add_prefix('level_'))
bac = bac.join(bac['classification'].str.split(';', expand=True).add_prefix('level_'))
#arc.drop(columns=['classification'], inplace=True)
#bac.drop(columns=['classification'], inplace=True)

In [ ]:
#Split the target name to retrieve the bin and create a new column with bin name  
pet_new['Bin'] = pet_new['target_name'].str.split('_').str[0:5].str.join('_')

In [15]:
#Concatenate the arc and bac dataframes
phylo = pd.concat([arc, bac])
phylo.head()

,user_genome,classification,level_0,level_1,level_2,level_3,level_4,level_5,level_6
0,D4991_C11_H1_Bin_10,d__Archaea;p__Halobacteriota;c__Syntropharchae...,d__Archaea,p__Halobacteriota,c__Syntropharchaeia,o__Syntropharchaeales,f__Syntropharchaeaceae,g__Syntropharchaeum_A,s__
1,D4991_C11_H1_Bin_108,d__Archaea;p__Thermoplasmatota;c__E2;o__DHVEG-...,d__Archaea,p__Thermoplasmatota,c__E2,o__DHVEG-1,f__B8-G13,g__B8-G13,s__B8-G13 sp021162485
2,D4991_C11_H1_Bin_113,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__Iainarchaeales,f__JAFCCW01,g__JAFCCW01,s__
3,D4991_C11_H1_Bin_119,d__Archaea;p__Altiarchaeota;c__Altiarchaeia;o_...,d__Archaea,p__Altiarchaeota,c__Altiarchaeia,o__IMC4,f__,g__,s__
4,D4991_C11_H1_Bin_12,d__Archaea;p__Thermoproteota;c__Bathyarchaeia;...,d__Archaea,p__Thermoproteota,c__Bathyarchaeia,o__B26-1,f__UBA233,g__PIYA01,s__


In [ ]:
#Join arc and bac to pet_new
pet_new = pet_new.merge(phylo, left_on='Bin', right_on='user_genome', how='left')
pet_new.head()

,target_name,query_name,full_sequence_evalue,full_sequence_score,full_sequence_bias,domain_e_evalue,domain_score,domain_bias,dive,ProteinID,...,Bin,user_genome,classification,level_0,level_1,level_2,level_3,level_4,level_5,level_6
0,D4998_C1112_H3_Bin_13_scaffold_290638_2,00129_Cutinase_Thermobifida_fusca_PET_muscle,1.900000e-08,43.0,0.0,3.300000e-08,42.3,0.0,D4998_C1112_H3,D4998_C1112_H3_Bin_13_scaffold_290638_2,...,D4998_C1112_H3_Bin_13,D4998_C1112_H3_Bin_13,d__Bacteria;p__Planctomycetota;c__Planctomycet...,d__Bacteria,p__Planctomycetota,c__Planctomycetia,o__Pirellulales,f__Pirellulaceae,g__UWMA-0346,s__UWMA-0346 sp012959465
1,D4998_C1112_H3_Bin_22_scaffold_55156_3,00129_Cutinase_Thermobifida_fusca_PET_muscle,1.500000e-08,43.4,0.0,8.600000e-02,21.2,0.0,D4998_C1112_H3,D4998_C1112_H3_Bin_22_scaffold_55156_3,...,D4998_C1112_H3_Bin_22,D4998_C1112_H3_Bin_22,d__Archaea;p__Thermoproteota;c__Thermoprotei;o...,d__Archaea,p__Thermoproteota,c__Thermoprotei,o__Thermofilales,f__,g__,s__
2,D4998_C1112_H3_Bin_150_scaffold_19950_40,00129_Cutinase_Thermobifida_fusca_PET_muscle,3.400000e-06,35.7,0.0,1.500000e-02,23.7,0.0,D4998_C1112_H3,D4998_C1112_H3_Bin_150_scaffold_19950_40,...,D4998_C1112_H3_Bin_150,D4998_C1112_H3_Bin_150,d__Archaea;p__Thermoproteota;c__Thermoprotei_A...,d__Archaea,p__Thermoproteota,c__Thermoprotei_A,o__Sulfolobales,f__Desulfurococcaceae,g__JAGGXK01,s__
3,D4998_C1112_H3_Bin_41_scaffold_101953_3,00129_Cutinase_Thermobifida_fusca_PET_muscle,2.200000e-08,42.8,1.8,3.200000e-08,42.3,1.8,D4998_C1112_H3,D4998_C1112_H3_Bin_41_scaffold_101953_3,...,D4998_C1112_H3_Bin_41,D4998_C1112_H3_Bin_41,d__Bacteria;p__Planctomycetota;c__UBA1135;o__U...,d__Bacteria,p__Planctomycetota,c__UBA1135,o__UBA1135,f__GCA-002686595,g__DUCM01,s__DUCM01 sp012959785
4,D4998_C1112_H3_Bin_198_scaffold_234535_28,00129_Cutinase_Thermobifida_fusca_PET_muscle,1.800000e-08,43.2,0.0,1.200000e-02,24.0,0.0,D4998_C1112_H3,D4998_C1112_H3_Bin_198_scaffold_234535_28,...,D4998_C1112_H3_Bin_198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
#Sort on lowest e-value and highest temperature and then choose the top unique columns by classification
pet_new = pet_new.sort_values(by=['full_sequence_evalue', 'temperature'], ascending=[True, False])
pet_new_ = pet_new.drop_duplicates(subset=['classification'], keep='first')
pet_new.shape 

(363, 23)

In [47]:
pet_new_.shape

(196, 23)

In [48]:
pet_new_

,target_name,query_name,full_sequence_evalue,full_sequence_score,full_sequence_bias,domain_e_evalue,domain_score,domain_bias,dive,ProteinID,...,Bin,user_genome,classification,level_0,level_1,level_2,level_3,level_4,level_5,level_6
272,D4993_C5_H1_Bin_524_scaffold_590470_15,00129_Cutinase_Thermobifida_fusca_PET_muscle,1.600000e-42,154.8,7.8,3.500000e-12,55.3,0.3,D4993_C5_H1,D4993_C5_H1_Bin_524_scaffold_590470_15,...,D4993_C5_H1_Bin_524,D4993_C5_H1_Bin_524,d__Bacteria;p__Chloroflexota;c__Anaerolineae;o...,d__Bacteria,p__Chloroflexota,c__Anaerolineae,o__JAFGEY01,f__JAFGEY01,g__,s__
158,D4993_C5_H4_Bin_238_scaffold_20386_9,00129_Cutinase_Thermobifida_fusca_PET_muscle,2.100000e-20,82.3,0.1,2.900000e-20,81.8,0.1,D4993_C5_H4,D4993_C5_H4_Bin_238_scaffold_20386_9,...,D4993_C5_H4_Bin_238,D4993_C5_H4_Bin_238,d__Archaea;p__Thermoproteota;c__Bathyarchaeia;...,d__Archaea,p__Thermoproteota,c__Bathyarchaeia,o__B26-1,f__WUQV01,g__,s__
332,D4998_C2223_H2_Bin_329_scaffold_361784_9,00129_Cutinase_Thermobifida_fusca_PET_muscle,3.700000e-18,74.9,2.3,2.300000e-17,72.3,2.3,D4998_C2223_H2,D4998_C2223_H2_Bin_329_scaffold_361784_9,...,D4998_C2223_H2_Bin_329,D4998_C2223_H2_Bin_329,d__Bacteria;p__Chloroflexota;c__Anaerolineae;o...,d__Bacteria,p__Chloroflexota,c__Anaerolineae,o__UBA1429,f__UBA1429,g__,s__
212,D4993_C5_H3_Bin_412_scaffold_75643_3,00129_Cutinase_Thermobifida_fusca_PET_muscle,3.900000e-17,71.5,1.9,1.000000e-16,70.2,1.9,D4993_C5_H3,D4993_C5_H3_Bin_412_scaffold_75643_3,...,D4993_C5_H3_Bin_412,D4993_C5_H3_Bin_412,d__Bacteria;p__Chloroflexota;c__Anaerolineae;o...,d__Bacteria,p__Chloroflexota,c__Anaerolineae,o__JADYZS01,f__JADYZS01,g__,s__
299,D4993_C5_H1_Bin_313_scaffold_691639_41,00129_Cutinase_Thermobifida_fusca_PET_muscle,1.700000e-14,62.9,0.0,3.400000e-14,61.9,0.0,D4993_C5_H1,D4993_C5_H1_Bin_313_scaffold_691639_41,...,D4993_C5_H1_Bin_313,D4993_C5_H1_Bin_313,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__GWF2-38-335,g__,s__
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,D4993_C5_H1_Bin_235_scaffold_166170_7,00129_Cutinase_Thermobifida_fusca_PET_muscle,4.000000e-05,32.2,0.0,6.100000e-05,31.5,0.0,D4993_C5_H1,D4993_C5_H1_Bin_235_scaffold_166170_7,...,D4993_C5_H1_Bin_235,D4993_C5_H1_Bin_235,d__Bacteria;p__Planctomycetota;c__Phycisphaera...,d__Bacteria,p__Planctomycetota,c__Phycisphaerae,o__,f__,g__,s__
126,D4994_C39_H1_Bin_146_scaffold_636346_1,00129_Cutinase_Thermobifida_fusca_PET_muscle,4.200000e-05,32.1,0.4,7.300000e+00,14.9,0.0,D4994_C39_H1,D4994_C39_H1_Bin_146_scaffold_636346_1,...,D4994_C39_H1_Bin_146,D4994_C39_H1_Bin_146,d__Bacteria;p__Armatimonadota;c__UBA5377;o__UB...,d__Bacteria,p__Armatimonadota,c__UBA5377,o__UBA5377,f__JABUFB01,g__JABUFB01,s__
321,D4998_C2223_H2_Bin_339_scaffold_221503_1,00129_Cutinase_Thermobifida_fusca_PET_muscle,4.300000e-05,32.0,0.0,5.800000e-05,31.6,0.0,D4998_C2223_H2,D4998_C2223_H2_Bin_339_scaffold_221503_1,...,D4998_C2223_H2_Bin_339,D4998_C2223_H2_Bin_339,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,d__Bacteria,p__Pseudomonadota,c__Gammaproteobacteria,o__Woeseiales,f__Woeseiaceae,g__SZUA-117,s__
120,D4994_C39_H1_Bin_679_scaffold_476470_86,00129_Cutinase_Thermobifida_fusca_PET_muscle,4.500000e-05,32.0,0.0,3.100000e-02,22.6,0.0,D4994_C39_H1,D4994_C39_H1_Bin_679_scaffold_476470_86,...,D4994_C39_H1_Bin_679,D4994_C39_H1_Bin_679,d__Bacteria;p__Planctomycetota;c__DG-23;o__;f_...,d__Bacteria,p__Planctomycetota,c__DG-23,o__,f__,g__,s__


In [ ]:
#write out a fasta file of all of the proteins
pet_fp = '../../data/results/PETHits_uniqueclassification_Guaymas2020.fasta'
with open(pet_fp, 'w') as f:
    for seq in pet_new_[['target_name', 'aa_sequence']].values:
        f.write('>' + seq[0] + '\n')
        f.write(seq[1] + '\n')
    f.close()

#Choose candidates for further analysis from these unique phyla. 